<a href="https://colab.research.google.com/github/keisuke58/keisuke58/blob/main/biofilmbaysian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution, root
import sys
import pandas as pd

# 設定: 見やすいグラフ
np.seterr(all='ignore')
plt.rcParams.update({'font.size': 10, 'figure.dpi': 100, 'lines.linewidth': 2})

# =============================================================================
# 1. Physics Engine (Hierarchical Solver)
# =============================================================================
class HierarchicalBiofilmSolver:
    def __init__(self):
        self.Kp1 = 1e-4
        self.Eta_val = 1.0

    def _run_simulation(self, n_species, params_A, params_B,
                        initial_phi, alpha_val, c_val, dt, n_steps):

        A = np.array(params_A)
        b_diag = np.array(params_B)

        phi_vec = np.array([initial_phi] * n_species)
        phi0 = 1.0 - np.sum(phi_vec)
        psi_vec = np.array([0.999] * n_species)
        gamma = 1e-3
        g_current = np.concatenate([phi_vec, [phi0], psi_vec, [gamma]])

        traj_data = []
        # Sampling: 20 points
        sample_indices = np.linspace(n_steps // 20, n_steps, 20, dtype=int)

        for step in range(1, n_steps + 1):
            sol = root(self._residual_func, g_current,
                       args=(g_current, n_species, dt, A, b_diag, alpha_val, c_val),
                       method='lm', tol=1e-5)

            if not sol.success:
                sol = root(self._residual_func, g_current,
                           args=(g_current, n_species, dt, A, b_diag, alpha_val, c_val),
                           method='hybr', tol=1e-5)
                if not sol.success: return None

            g_new = sol.x
            g_new[0:n_species+1] = np.clip(g_new[0:n_species+1], 1e-6, 1.0-1e-6)
            g_new[n_species+1:2*n_species+1] = np.clip(g_new[n_species+1:2*n_species+1], 0.1, 5.0)
            g_current = g_new.copy()

            if step in sample_indices:
                # Biomass = phi * psi
                phi = g_current[0:n_species]
                psi = g_current[n_species+1:2*n_species+1]
                traj_data.append(phi * psi)

        return np.array(traj_data)

    def _residual_func(self, g_new, g_old, n, dt, A, b_diag, alpha, c_val):
        phi = g_new[0:n]; phi0 = g_new[n]; psi = g_new[n+1 : 2*n+1]; gamma = g_new[-1]
        phidot = (phi - g_old[0:n]) / dt
        phi0dot = (phi0 - g_old[n]) / dt
        psidot = (psi - g_old[n+1 : 2*n+1]) / dt

        Q = np.zeros_like(g_new)
        Eta_vec = np.full(n, self.Eta_val)

        CapitalPhi = phi * psi
        Interaction_dot = A @ CapitalPhi

        denom_phi = np.sign((phi-1)**3 * phi**3) * np.maximum(np.abs((phi-1)**3 * phi**3), 1e-12)
        Q[0:n] = (self.Kp1 * (2. - 4.*phi)) / denom_phi + (1./Eta_vec)*(gamma + (Eta_vec + Eta_vec*psi**2)*phidot + Eta_vec*phi*psi*psidot) - (c_val/Eta_vec) * psi * Interaction_dot

        denom_phi0 = np.sign((phi0-1)**3 * phi0**3) * np.maximum(np.abs((phi0-1)**3 * phi0**3), 1e-12)
        Q[n] = gamma + (self.Kp1*(2.-4.*phi0))/denom_phi0 + phi0dot

        denom_psiA = np.sign((psi-1)**2 * psi**3) * np.maximum(np.abs((psi-1)**2 * psi**3), 1e-12)
        denom_psiB = np.sign((psi-1)**3 * psi**2) * np.maximum(np.abs((psi-1)**3 * psi**2), 1e-12)
        Q[n+1 : 2*n+1] = (-2.*self.Kp1)/denom_psiA - (2.*self.Kp1)/denom_psiB + (b_diag * alpha / Eta_vec) * psi + phi*psi*phidot + phi**2*psidot - (c_val/Eta_vec) * phi * Interaction_dot

        Q[-1] = np.sum(phi) + phi0 - 1.0
        return Q

    # --- Wrappers (Table 3) ---
    def run_M1(self, params):
        p_a11, p_a12, p_a22, p_b1, p_b2 = params
        A = [[p_a11, p_a12], [p_a12, p_a22]]
        B = [p_b1, p_b2]
        return self._run_simulation(2, A, B, 0.2, 100.0, 100.0, 1e-5, 2500)

    def run_M2(self, params):
        p_a33, p_a34, p_a44, p_b3, p_b4 = params
        A = [[p_a33, p_a34], [p_a34, p_a44]]
        B = [p_b3, p_b4]
        return self._run_simulation(2, A, B, 0.2, 10.0, 100.0, 1e-5, 5000)

    def run_M3(self, params, known_M1, known_M2):
        a13, a14, a23, a24 = params
        a11, a12, a22, b1, b2 = known_M1
        a33, a34, a44, b3, b4 = known_M2
        A = [[a11, a12, a13, a14], [a12, a22, a23, a24], [a13, a23, a33, a34], [a14, a24, a34, a44]]
        B = [b1, b2, b3, b4]
        return self._run_simulation(4, A, B, 0.02, 0.0, 25.0, 1e-4, 750)

    def run_M3_val(self, est_M1, est_M2, est_M3):
        a11, a12, a22, b1, b2 = est_M1
        a33, a34, a44, b3, b4 = est_M2
        a13, a14, a23, a24 = est_M3
        A = [[a11, a12, a13, a14], [a12, a22, a23, a24], [a13, a23, a33, a34], [a14, a24, a34, a44]]
        B = [b1, b2, b3, b4]

        # Validation run (N=1500, dt=1e-4)
        dt = 1e-4; n_steps = 1500; c_val = 25.0
        g_current = np.concatenate([[0.02]*4, [0.92], [0.999]*4, [1e-3]])
        traj = []; t_axis = []

        for step in range(1, n_steps + 1):
            alpha = 50.0 if step > 500 else 0.0
            sol = root(self._residual_func, g_current, args=(g_current, 4, dt, A, b_diag, alpha, c_val), method='lm', tol=1e-5)
            if not sol.success: return None, None
            g_new = sol.x
            g_new[0:9] = np.clip(g_new[0:9], 1e-6, 5.0)
            g_current = g_new.copy()
            if step % 10 == 0:
                traj.append(g_current[0:4] * g_current[5:9])
                t_axis.append(step/1500.0) # Normalized time
        return np.array(t_axis), np.array(traj)

# =============================================================================
# Optimization Logic
# =============================================================================
solver = HierarchicalBiofilmSolver()

def obj_M1(params, data):
    sim = solver.run_M1(params)
    if sim is None: return 1e15
    return np.mean((sim - data)**2)

def obj_M2(params, data):
    sim = solver.run_M2(params)
    if sim is None: return 1e15
    return np.mean((sim - data)**2)

def obj_M3(params, data, m1, m2):
    sim = solver.run_M3(params, m1, m2)
    if sim is None: return 1e15
    return np.mean((sim - data)**2)

def plot_fit(ax, data, fit, title, labels):
    steps = np.arange(len(data))
    colors = ['r', 'b', 'g', 'orange']
    for i in range(data.shape[1]):
        ax.plot(steps, data[:, i], '.', color=colors[i], alpha=0.4, label=f'{labels[i]} Data')
        ax.plot(steps, fit[:, i], '-', color=colors[i], linewidth=2, label=f'{labels[i]} Fit')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)

# =============================================================================
# MAIN EXECUTION
# =============================================================================
if __name__ == "__main__":
    print("=== Case II: Fast Optimization & All Figures ===")

    # True Parameters (Paper's values)
    TRUE_M1 = [2.0, 0.5, 1.5, 1.0, 2.0]
    TRUE_M2 = [1.2, 0.3, 1.8, 1.5, 0.5]
    TRUE_M3 = [0.2, 0.1, 0.1, 0.3]

    # 1. Generate Synthetic Data
    print("Generating Data...")
    d_M1 = solver.run_M1(TRUE_M1) + np.random.normal(0, 0.002, (20, 2))
    d_M2 = solver.run_M2(TRUE_M2) + np.random.normal(0, 0.002, (20, 2))
    d_M3 = solver.run_M3(TRUE_M3, TRUE_M1, TRUE_M2) + np.random.normal(0, 0.002, (20, 4))

    # Settings
    STRATEGY = 'best1bin'
    MAX_ITER = 20
    POP_SIZE = 10

    # --- Stage 1: M1 ---
    print("\nEstimating M1...")
    res1 = differential_evolution(obj_M1, [(0,3)]*5, args=(d_M1,), strategy=STRATEGY, maxiter=MAX_ITER, popsize=POP_SIZE, disp=True)
    est_M1 = res1.x

    # Fig 9: M1 Fit
    fig, ax = plt.subplots(figsize=(8, 5))
    plot_fit(ax, d_M1, solver.run_M1(est_M1), "Fig 9: M1 (Sp 1 & 2) Fit", ["Sp1", "Sp2"])
    plt.show()

    # --- Stage 2: M2 ---
    print("\nEstimating M2...")
    res2 = differential_evolution(obj_M2, [(0,3)]*5, args=(d_M2,), strategy=STRATEGY, maxiter=MAX_ITER, popsize=POP_SIZE, disp=True)
    est_M2 = res2.x

    # Fig 11: M2 Fit
    fig, ax = plt.subplots(figsize=(8, 5))
    plot_fit(ax, d_M2, solver.run_M2(est_M2), "Fig 11: M2 (Sp 3 & 4) Fit", ["Sp3", "Sp4"])
    plt.show()

    # --- Stage 3: M3 ---
    print("\nEstimating M3...")
    res3 = differential_evolution(obj_M3, [(0,3)]*4, args=(d_M3, est_M1, est_M2), strategy=STRATEGY, maxiter=MAX_ITER, popsize=POP_SIZE, disp=True)
    est_M3 = res3.x

    # Fig 13: M3 Fit
    fig, ax = plt.subplots(figsize=(8, 5))
    plot_fit(ax, d_M3, solver.run_M3(est_M3, est_M1, est_M2), "Fig 13: M3 (All Species) Fit", ["Sp1", "Sp2", "Sp3", "Sp4"])
    plt.show()

    # --- PRINT ALL PARAMETERS ---
    print("\n=== FINAL ESTIMATED PARAMETERS ===")
    param_names = ["a11","a12","a22","b1","b2", "a33","a34","a44","b3","b4", "a13","a14","a23","a24"]
    all_est = np.concatenate([est_M1, est_M2, est_M3])
    all_true = np.concatenate([TRUE_M1, TRUE_M2, TRUE_M3])

    df_res = pd.DataFrame({"Parameter": param_names, "True": all_true, "Estimated": np.round(all_est, 4)})
    df_res["Error"] = df_res["Estimated"] - df_res["True"]
    print(df_res)

    # --- Fig 14: Bar Chart ---
    plt.figure(figsize=(12, 6))
    x = np.arange(len(all_true))
    plt.bar(x - 0.2, all_true, 0.4, label='True Value', color='#ff7f0e')
    plt.bar(x + 0.2, all_est, 0.4, label='Estimated Value', color='#1f77b4')
    plt.xticks(x, param_names)
    plt.title("Fig 14: Parameter Comparison")
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.show()

    # --- Fig 15: M3val Validation ---
    print("\nRunning Validation (M3val)...")
    t, val = solver.run_M3_val(est_M1, est_M2, est_M3)

    plt.figure(figsize=(10, 6))
    cols = ['r', 'b', 'g', 'orange']
    for i in range(4):
        plt.plot(t, val[:, i], color=cols[i], linewidth=2, label=f'Species {i+1}')
    plt.axvline(x=0.333, color='k', linestyle='--', label='Antibiotics ON')
    plt.title("Fig 15: Validation with Antibiotics Shock")
    plt.xlabel("Normalized Time")
    plt.ylabel("Living Biomass")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

=== Case II: Fast Optimization & All Figures ===
Generating Data...

Estimating M1...
